**My Understanding -**

There are three excel files given, named as the type of light illuminated on the device.
There are 5 possible initial conductance states and 4 different mask-sets(I1, I2, I3, I4) that could be applied onto the device.

In the given data of 365nm optical illumination, for example, for I1 mask, 0th initial conductance state if we apply "0111" optical pulse train, then the corresponding final conductance is 3.38E-10 S.

"0111" impulse train means first we have applied a "1" impulse 3 times and then applied a "0" impulse.


**My Understanding 2**

There are three excel files given, named as the type of light illuminated on the device. There are 5 possible different optical masks(0, 1, 2, 3, 4) and 4 different conductance masks(I1, I2, I3, I4) that could be applied on.

In the given data of 365nm wavelength light, for example, I1 current mask, 0th row (0th optical mask) if we apply "0111" optical pulse train on the given device, then the corresponding final conductance of the device is 3.38E-10 S

**0111** impulse train means first we apply a **1** impulse 3 times and then apply a **0** impulse

In [2]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm


In [3]:
class BinarizeTransform:
    def __call__(self, img):
        return (img>0.5).float()
transform = transforms.Compose([
    transforms.ToTensor(),
    BinarizeTransform()
])
mnist_data_train = datasets.MNIST(root="data/MNIST", train=True, download=True, transform=transform)
mnist_data_test = datasets.MNIST(root="data/MNIST", train=False, download=True, transform=transform)


In [4]:
json_data = {
    "365nm":{
        "I1":range(0, 5),
        "I2":range(10, 15),
        "I3":range(18, 23),
        "I4":range(25, 30)
    },
    "455nm":{
        "I1":range(0, 5),
        "I2":range(7, 12),
        "I3":range(14, 19),
        "I4":range(21, 26)
    },
    "White":{
        "I1":range(0, 5),
        "I2":range(9, 14),
        "I3":range(16, 21),
        "I4":range(24, 29)
    }
}

In [5]:
filename = "455nm"
path = "data/"+filename+".xlsx"

df = pd.read_excel(path, usecols='B:Q')
tables = [df.iloc[json_data[filename][key]].copy().reset_index(drop=True) for key in list(json_data[filename].keys())]
# tables[0].insert(0, "opticalmask", "I1")
for idx, table in enumerate(tables):
    table.insert(0, "mask", idx)
    # table['conductance_state'] = table.index


# device_indices = torch.randint(0, 5, (196,))
# mask_sets = [0, 1, 2, 3]

In [71]:
NUMBER_OF_MASK_SETS = 4
mask_sets = np.random.randint(0, 20, (3,))
mask_sets = np.array([(n//5, n%5) for n in mask_sets])  # [ (current_mask, optical_mask) ]


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, mnist_data, tables, mask_sets):

        self.processed_data = []
        self.labels = []

        for idx in range(len(mnist_data)):
            image, label = mnist_data[idx]
            image = image.reshape(1, 196, 4)
            image_combined = (
                image[:, :, 0] * 1000 + 
                image[:, :, 1] * 100 + 
                image[:, :, 2] * 10 + 
                image[:, :, 3]
            )
            x = [ tables[current_mask].loc[optical_mask][image_combined[0].tolist()].tolist()
                 for current_mask, optical_mask in mask_sets
                 ]
            x = torch.tensor(x).reshape((1, 196 * len(mask_sets)))
            self.processed_data.append(x)
            self.labels.append(label)

            if (idx+1)%1000 == 0:
                print(f"{(idx+1)}/{len(mnist_data)} done")

        
        self.processed_data = torch.cat(self.processed_data, axis = 0)
        self.labels = torch.tensor(self.labels)
                
    def __len__(self):
        return self.processed_data.shape[0]
    
    def __getitem__(self, index):
        return self.processed_data[index], self.labels[index]

In [73]:
train_dataset = CustomDataset(mnist_data_train, tables, mask_sets )

1000/60000 done
2000/60000 done
3000/60000 done
4000/60000 done
5000/60000 done
6000/60000 done
7000/60000 done
8000/60000 done
9000/60000 done
10000/60000 done
11000/60000 done
12000/60000 done
13000/60000 done
14000/60000 done
15000/60000 done
16000/60000 done
17000/60000 done
18000/60000 done
19000/60000 done
20000/60000 done
21000/60000 done
22000/60000 done
23000/60000 done
24000/60000 done
25000/60000 done
26000/60000 done
27000/60000 done
28000/60000 done
29000/60000 done
30000/60000 done
31000/60000 done
32000/60000 done
33000/60000 done
34000/60000 done
35000/60000 done
36000/60000 done
37000/60000 done
38000/60000 done
39000/60000 done
40000/60000 done
41000/60000 done
42000/60000 done
43000/60000 done
44000/60000 done
45000/60000 done
46000/60000 done
47000/60000 done
48000/60000 done
49000/60000 done
50000/60000 done
51000/60000 done
52000/60000 done
53000/60000 done
54000/60000 done
55000/60000 done
56000/60000 done
57000/60000 done
58000/60000 done
59000/60000 done
60000/

In [6]:

device_indices = torch.randint(0, 5, (196,))
mask_sets = [0, 1, 2, 3]

In [67]:
tables[0].columns.get_indexer([1010, 1011, 0000])

array([ 7, 13, 16], dtype=int64)

In [9]:
mask_sets

[0, 1, 2, 3]

In [86]:

pd.concat(tables, axis = 0)

,mask,1000,100,10,1,1100,110,1010,1001,101,11,1110,1101,1011,111,1111,0
0,0,3.009090e-10,2.903490e-10,2.883020e-10,2.771180e-10,2.933200e-10,2.965650e-10,2.947180e-10,2.832840e-10,2.793650e-10,2.906240e-10,2.836090e-10,2.892760e-10,2.789900e-10,2.739230e-10,2.863550e-10,3.022600e-10
1,0,1.588410e-09,2.790150e-10,2.840330e-10,2.928210e-10,1.576930e-09,2.852810e-10,1.570990e-09,1.575830e-09,2.863300e-10,2.819610e-10,1.544350e-09,1.508550e-09,1.484510e-09,2.843080e-10,1.452630e-09,3.050800e-10
2,0,6.543270e-10,1.388720e-09,2.856810e-10,2.899500e-10,1.800560e-09,1.384780e-09,6.583710e-10,6.576220e-10,1.347830e-09,2.928700e-10,1.763440e-09,1.735250e-09,6.543020e-10,1.273610e-09,1.683230e-09,3.095980e-10
3,0,4.803520e-10,5.823050e-10,1.581600e-09,2.916970e-10,7.546080e-10,1.904030e-09,1.814960e-09,4.855190e-10,5.844520e-10,1.532920e-09,2.076960e-09,7.692120e-10,1.723290e-09,1.796340e-09,1.998470e-09,3.180350e-10
4,0,4.032870e-10,5.069140e-10,6.566240e-10,1.585420e-09,5.507510e-10,8.960490e-10,7.723330e-10,1.732380e-09,1.822270e-09,1.954740e-09,9.989510e-10,1.894215e-09,2.019280e-09,2.131680e-09,2.250160e-09,3.140410e-10
0,1,4.134090e-10,4.028490e-10,4.008020e-10,3.896180e-10,4.058200e-10,4.090650e-10,4.072180e-10,3.957840e-10,3.918650e-10,4.031240e-10,3.961090e-10,4.017760e-10,3.914900e-10,3.864230e-10,3.988550e-10,3.022600e-10
1,1,1.700910e-09,3.915150e-10,3.965330e-10,4.053210e-10,1.689430e-09,3.977810e-10,1.683490e-09,1.688330e-09,3.988300e-10,3.944610e-10,1.656850e-09,1.621050e-09,1.597010e-09,3.968080e-10,1.565130e-09,3.050800e-10
2,1,7.668270e-10,1.501220e-09,3.981810e-10,4.024500e-10,1.913060e-09,1.497280e-09,7.708710e-10,7.701220e-10,1.460330e-09,4.053700e-10,1.875940e-09,1.847750e-09,7.668020e-10,1.386110e-09,1.795730e-09,3.095980e-10
3,1,5.928520e-10,6.948050e-10,1.694100e-09,4.041970e-10,8.671080e-10,2.016530e-09,1.927460e-09,5.980190e-10,6.969520e-10,1.645420e-09,2.189460e-09,8.817120e-10,1.835790e-09,1.908840e-09,2.110970e-09,3.180350e-10
4,1,5.157870e-10,5.494140e-10,7.691240e-10,1.697920e-09,6.632510e-10,9.085490e-10,8.848330e-10,1.844880e-09,1.915480e-09,2.067240e-09,1.011450e-09,1.954650e-09,2.145270e-09,2.244180e-09,2.562660e-09,3.140410e-10


In [7]:
class CustomMNISTDataset(Dataset):
    def __init__(self, mnist_data, tables, device_indices , mask_sets = mask_sets):
        self.mask_sets = mask_sets
        self.processed_data = []
        self.labels = []
        self.combined_table = pd.concat(tables, axis = 0)
        self.device_indices = list(map(int, device_indices))
        
        for idx in tqdm(range(len(mnist_data))):
            image, label = mnist_data[idx]
            image = image.reshape(1, 196, 4)
            image_combined = (
                image[:, :, 0] * 1000 + 
                image[:, :, 1] * 100 + 
                image[:, :, 2] * 10 + 
                image[:, :, 3]
            )
            table_indices = [device_indices + mask*5 for mask in mask_sets]
            table_indices = torch.cat(table_indices, axis = 0)
            
            names_of_columns_needed = image_combined[0].repeat(len(mask_sets)).int().tolist()
            column_indices = self.combined_table.columns.get_indexer(names_of_columns_needed)
            
            # print(self.combined_table.iloc[table_indices].values[np.arange(len(table_indices)), column_indices])

            # print(table_indices.shape)
            # print(image_combined[0].repeat(len(mask_sets)).shape)
            # print(image_combined[0].repeat(len(mask_sets)).int())
            # print(self.combined_table.iloc[table_indices].apply(lambda row : row[column_indices[row.name]], axis=1) )

            # y = self.combined_table.iloc[table_indices] \
            #         .to_numpy()[range(len(table_indices)), column_indices ]
            x = self.combined_table.iloc[table_indices].values[np.arange(len(table_indices)), column_indices]
            

            # x = [
            #     tables[mask].loc[int(device), int(image_combined[0][i])]
            #     for mask in self.mask_sets
            #     for i, device in enumerate(self.device_indices)
            # ]
            
            self.processed_data.append(x)
            self.labels.append(label)
            
        self.processed_data = torch.tensor(self.processed_data)
        self.labels = torch.tensor(self.labels)

    def __len__(self):
        return len(self.processed_data)

    def __getitem__(self, idx):
        return self.processed_data[idx], self.labels[idx]



In [8]:
custom_mnist_dataset = CustomMNISTDataset(mnist_data_train, tables=tables, mask_sets=mask_sets, device_indices=device_indices)

100%|██████████| 60000/60000 [00:47<00:00, 1269.67it/s]
C:\Users\adith\AppData\Local\Temp\ipykernel_3400\967199193.py:45: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  self.processed_data = torch.tensor(self.processed_data)


In [ ]:
custom_mnist_dataset_test = CustomMNISTDataset(mnist_data_test, tables=tables, mask_sets=mask_sets, device_indices=device_indices)

In [ ]:
data_loader = DataLoader(custom_mnist_dataset, batch_size=32, shuffle=True)

for images, labels in data_loader:
    print(f"Image batch shape: {images.shape}")
    print(f"Label batch shape: {labels.shape}")
    break

In [ ]:
# np.savez("data/processed/"+filename+".npz",
#         preprocessed_train_data = custom_mnist_dataset.processed_data,
#         train_labels = custom_mnist_dataset.labels,
#         processed_test_labels = custom_mnist_dataset_test.processed_data,
#         test_labels = custom_mnist_dataset_test.labels
#         )